## Example Code

In [1]:
def plot_point_clouds(point_clouds, titles=None, point_size=1, color_map='gray'):
    """
    Plots multiple 3D point clouds in separate subplots.

    Parameters:
    - point_clouds: List of 3D point cloud arrays (each Nx3 or Nx4 if with intensity)
    - titles: List of titles for each subplot (optional)
    - point_size: Size of points in the scatter plot (optional, default=1)
    - color_map: Color map to use for point intensity (optional, default='gray')
    """
    num_clouds = len(point_clouds)
    fig = plt.figure(figsize=(5 * num_clouds, 5))
    
    for i, cloud in enumerate(point_clouds):
        ax = fig.add_subplot(1, num_clouds, i + 1, projection='3d')
        
        # Downsample points if needed for visualization
        step = max(1, cloud.shape[0] // 1000)  # Adjust to control display density
        sample_cloud = cloud[::step]
        
        if sample_cloud.shape[1] == 3:
            ax.scatter(sample_cloud[:, 0], sample_cloud[:, 1], sample_cloud[:, 2], 
                       s=point_size, c='black')  # No intensity values
        elif sample_cloud.shape[1] >= 4:
            ax.scatter(sample_cloud[:, 0], sample_cloud[:, 1], sample_cloud[:, 2], 
                       s=point_size, c=sample_cloud[:, 3], cmap=color_map)  # Use intensity for color

        # Set title if provided
        if titles and i < len(titles):
            ax.set_title(titles[i])
            
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')

    plt.show()


In [2]:
import numpy as np
from scipy.spatial import cKDTree

def chamfer_distance(point_cloud1, point_cloud2):
    """
    Computes the Chamfer Distance between two point clouds.

    Parameters:
    - point_cloud1: Nx3 numpy array of points (first point cloud)
    - point_cloud2: Mx3 numpy array of points (second point cloud)

    Returns:
    - chamfer_dist: Chamfer Distance between point_cloud1 and point_cloud2
    """
    # Build KD-trees for fast nearest-neighbor search
    tree1 = cKDTree(point_cloud1)
    tree2 = cKDTree(point_cloud2)
    
    # Nearest neighbor distances from each point in point_cloud1 to point_cloud2
    distances1, _ = tree1.query(point_cloud2)
    # Nearest neighbor distances from each point in point_cloud2 to point_cloud1
    distances2, _ = tree2.query(point_cloud1)
    
    # Average distances for both directions
    chamfer_dist = np.mean(distances1**2) + np.mean(distances2**2)
    return chamfer_dist


In [3]:
import numpy as np
from scipy.spatial import cKDTree
from scipy.sparse import csgraph

def graph_wavelet_transform(point_cloud, k=10):
    """
    Performs a graph wavelet transform on a point cloud using the graph Laplacian.

    Parameters:
    - point_cloud: Nx3 numpy array of points (e.g., [x, y, z])
    - k: Number of nearest neighbors to use for graph construction

    Returns:
    - transformed: Graph wavelet transformed features (flattened array)
    """
    # Step 1: Construct a k-NN graph from the point cloud
    tree = cKDTree(point_cloud)
    distances, indices = tree.query(point_cloud, k=k+1)  # k+1 because query includes the point itself
    N = point_cloud.shape[0]

    # Create adjacency matrix (N x N) based on distances
    adjacency_matrix = np.zeros((N, N))
    for i in range(N):
        for j in range(1, k+1):  # Skip the first result, which is the point itself
            adjacency_matrix[i, indices[i, j]] = np.exp(-distances[i, j]**2)

    # Step 2: Compute the graph Laplacian
    laplacian = csgraph.laplacian(adjacency_matrix, normed=True)

    # Step 3: Perform the wavelet transform on each point's coordinates
    # This uses the eigenvalues and eigenvectors of the Laplacian
    eigvals, eigvecs = np.linalg.eigh(laplacian)
    wavelet_coeffs = eigvecs @ np.diag(np.exp(-eigvals)) @ eigvecs.T @ point_cloud

    # Flatten wavelet coefficients to produce a transformed array
    transformed = wavelet_coeffs.flatten()
    return transformed


In [4]:
import numpy as np
from scipy.sparse import csgraph
from scipy.spatial import cKDTree

def inverse_graph_wavelet_transform(transformed, point_cloud_shape, k=10):
    """
    Inverts the graph wavelet transformation on a point cloud, returning it to its original domain.

    Parameters:
    - transformed: Flattened wavelet-transformed point cloud data
    - point_cloud_shape: Tuple (N, 3) representing the original shape of the point cloud
    - k: Number of nearest neighbors used in graph construction

    Returns:
    - original_point_cloud: Reconstructed point cloud (Nx3 array)
    """
    # Reshape the flattened transformed data to match the original point cloud shape
    transformed = transformed.reshape(point_cloud_shape)

    # Step 1: Reconstruct the graph used in the transform (k-NN adjacency)
    tree = cKDTree(transformed)
    distances, indices = tree.query(transformed, k=k+1)
    N = point_cloud_shape[0]

    # Create adjacency matrix
    adjacency_matrix = np.zeros((N, N))
    for i in range(N):
        for j in range(1, k+1):
            adjacency_matrix[i, indices[i, j]] = np.exp(-distances[i, j]**2)

    # Step 2: Compute the Laplacian matrix
    laplacian = csgraph.laplacian(adjacency_matrix, normed=True)

    # Step 3: Invert the transformation using Laplacian's eigenvalues/eigenvectors
    eigvals, eigvecs = np.linalg.eigh(laplacian)
    inverse_wavelet_coeffs = eigvecs @ np.diag(np.exp(eigvals)) @ eigvecs.T @ transformed

    # Return the reconstructed point cloud in the original domain
    return inverse_wavelet_coeffs


In [5]:
def recover_l1(transformed, lam):
    """
    Recover sparse representation using L1 regularization for a transformed point cloud.

    Parameters:
    - transformed: The transformed point cloud data (e.g., after wavelet or graph wavelet transform)
    - lam: Regularization parameter (lambda) for controlling sparsity

    Returns:
    - transformed_recovered: Recovered sparse representation of the point cloud data
    """
    # Apply L1 regularization (soft thresholding) to each dimension of the transformed point cloud
    transformed_recovered = np.sign(transformed) * np.maximum(np.abs(transformed) - lam, 0)
    return transformed_recovered

In [6]:
def compressive_sensing_point_cloud(
        point_cloud, undersampling_ratio, iter, lam, lam_decay, error_function, 
        plot_process=False, plot_error=False):
    """
    Compressive sensing on LiDAR point cloud data.
    
    INPUT:
    - point_cloud: Original 3D point cloud data (N x 3 array where N is the number of points)
    - undersampling_ratio: Ratio of points to retain during undersampling
    - iter: Number of iterations to update the reconstruction
    - lam: Regularization parameter for L1 recovery
    - lam_decay: Decay factor for lam at each iteration
    - error_function: Function for calculating reconstruction error
    - plot_process: Boolean to plot the point cloud reconstruction at each iteration
    - plot_error: Boolean to plot the error curve over iterations

    OUTPUT:
    - Plots reconstructed point clouds and error curve if requested.
    """

    errors = []

    # Step 1: Randomly undersample the point cloud
    num_points = int(len(point_cloud) * undersampling_ratio)
    sampled_indices = np.random.choice(len(point_cloud), num_points, replace=False)
    point_cloud_undersampled = point_cloud[sampled_indices]

    # Initialize the reconstructed point cloud with the undersampled data
    point_cloud_reconstructed = point_cloud_undersampled.copy()

    if plot_process:
        plot_point_clouds([point_cloud, point_cloud_undersampled], 
                          titles=['Original', 'Undersampled'])

    for i in range(iter):
        # Apply graph or wavelet transform on the point cloud
        transformed_reconstruction = graph_wavelet_transform(point_cloud_reconstructed)

        # Recover sparse representation using L1 regularization
        transformed_recovered = recover_l1(transformed_reconstruction, lam)

        # Inverse transform to return to the 3D spatial domain
        point_cloud_reconstructed = inverse_graph_wavelet_transform(transformed_recovered, point_cloud_undersampled.shape)

        # Reinforce known (undersampled) points in the reconstruction
        point_cloud_reconstructed[sampled_indices] = point_cloud_undersampled

        # Compute error between original and reconstructed point clouds
        error = error_function(point_cloud, point_cloud_reconstructed)
        errors.append(error)

        # Decay lambda
        lam *= lam_decay

        # Plot every 5 iterations if plot_process is enabled
        if plot_process and (i + 1) % 5 == 0:
            plot_point_clouds([point_cloud, point_cloud_reconstructed], 
                              titles=['Original', f'Reconstructed (Iter {i + 1})'])

    # Plot the reconstruction result
    if not plot_process:
        plot_point_clouds([point_cloud, point_cloud_undersampled, point_cloud_reconstructed],
                          titles=['Original', 'Undersampled', 'Reconstructed'])

    # Plot the error convergence
    if plot_error:
        plt.figure(figsize=(10, 5))
        plt.plot(errors)
        plt.xlabel('Iteration')
        plt.title('Error Function')
        plt.show()

# Example usage:
compressive_sensing_point_cloud(
    point_cloud=point_cloud[:, :3], undersampling_ratio=0.3, iter=1, lam=150, 
    lam_decay=0.9, error_function=chamfer_distance, plot_process=False, plot_error=True)


NameError: name 'point_cloud' is not defined